In [2]:
import os 
from os import path 
import shutil
import csv
from PIL import Image
import numpy as np
import tabulate 

#Step-1

#Create a directory function
def create_directory(path,directory_name):

    directory_dest = path + "/" + directory_name
    os.mkdir(directory_dest)

# Create a database (type "dictionary") to memorazie all the data of the file I'm interested 
# with these characteristcs : 
# {"filename": {"type":"name directory", "size": "size of the file"}.
def create_dictionary_files(list_files,resource):

    dictionary_files={}
    # The file "recap.csv" must to be outside of the directory we will create so in order to do that 
    # I just do not save it in the dictionary.
    for name in list_files:
        if name == "recap.csv":
            continue
        

        resource_file = resource + "/" + name

        # Check to control if what we are analyzing it is a file. 
        if path.isfile(resource_file) :
            size = path.getsize(resource_file)
            name_split = path.splitext(name)

            #Check what type of file it is to put the file in the right directory
            if name_split[1] == ".jpg" or name_split[1] == ".png" or name_split[1] == ".jpeg":
                dictionary_files[name]= {"type" : "images","size":size}
            elif name_split[1] == ".txt" or name_split[1] == ".odt" or name_split[1] == ".docx":
                dictionary_files[name]= {"type" : "docs","size":size}
            elif name_split[1] == ".mp3":
                dictionary_files[name]= {"type" : "audio","size":size}
            else: 
                dictionary_files[name]= {"type" : "other","size":size}

    return dictionary_files

# I need to create a set to store all the directories already created.
# To not overwrite the macro directory with already existing directories.
def create_set_types(path_files):
    exist = False
    set_types = set()
    name_elements = os.listdir(path_files)
    name_files = list(name_elements)
    for name in name_elements: 
        if path.isdir(path_files + "/" + name):
            name_files.remove(name)
            set_types.add(name)

        # While i do this check it is usefull store if the "recap.csv" file it is already created.
        # In order to write a Hader ( we will do this in another part of the program) 
        # within the file if the file was not exist before.
        elif name == "recap.csv":
            exist = True 

    return set_types,name_files,exist

def iteration_files(position_files):
    # To check if the directory already exists, I create a set with the three categories (images, documents, audio).
    set_types,name_files, exist = create_set_types(position_files)
    # I create a dictionary to match each file to its type (image, document, audio) and size.
    dictionary_files = create_dictionary_files(name_files,position_files)

    # Check of the macro directory to manage the case wich it is empthy.
    if dictionary_files == None :
        exit("There are no free files on the macro directory")

    #Iteration of files
    for name,info in dictionary_files.items():
        print(f"{name} type:{info['type']} size:{info['size']}\n")
        # Check if the directory already exists.
        # If it does not exist, add the directory and move the files to the directory.
        if info["type"] not in set_types:
            set_types.add(info["type"])
            create_directory(position_files,info["type"])

            # move files to the right directory 
            resource = position_files + "/" + name
            dest_path = position_files + "/" + info["type"]
            shutil.move(resource,dest_path)

        else:
            # move files to the right directory
            resource = position_files + "/" + name
            dest_path = position_files + "/" + info["type"]
            shutil.move(resource,dest_path)
    
    return dictionary_files, exist

# Create a file CSV to recap all the file moved
def recap(path_position,dictionary,exist):

    name_path = path_position + "/" + "recap.csv"

    #Here is were we use the boolean "exist" the we made during the creating of the set().
    if not exist :
        with open(name_path,'w', encoding="UTF8", newline="") as f:
            writer = csv.writer(f)
            # Header
            writer.writerow(["name","type","size(B)"])
    
    with open(name_path,"a", encoding="UTF8", newline="") as f:
        writer = csv.writer(f)
        for name,info in dictionary.items():
            name_clean = path.splitext(name)[0]
            #Files  
            writer.writerow([name_clean, info["type"], str(info["size"])])

#Step-2

# This function it is useful to convert an image to an array with the Numpy library. 
def create_array_informations(name,directory):

    resource = directory + "/" + name
    im = Image.open(resource)
    a = np.array(im)
    return a

# Creation of a database with all the information we need about an image:
# 1) Height of the image, in pixels 
# 2) Width of the image, in pixels. 
# 3) If the image is greyscale, I'm interested in the average of the values in greyscale then of the one color layer. 
# 4) If the image is in color, I'm interested of the average of the values of each color layer 
def create_list_info(list,direct_images):
    # I chose to use a list because it was better to print with the "tabulate" form. 
    list_info = []
    # Iteration in the image directory
    for name in list:
        name_clean = os.path.splitext(name)[0]
        x = create_array_informations(name,direct_images)
        tipology = np.shape(x)

        # Find the mean values (grayscale, R,G,B,ALPHA) for each image.
        if len(tipology) == 2:
            grayscale = round(np.mean(x),2)
            # I insert within the table the values I'm interested in. 
            list_info.append([name_clean,tipology[0],tipology[1],grayscale,0.00,0.00,0.00,0.00])
        else:
            alpha = 0 
            tupla_averages = np.mean(x,axis=(0,1))
            if tipology[2] == 4 : 
                alpha = round(tupla_averages[3],2)
            R = round(tupla_averages[0],2)
            G = round(tupla_averages[1],2)
            B = round(tupla_averages[2],2)
            # I insert within the table the values that I am interested in.
            list_info.append([name_clean,tipology[0],tipology[1],0.00,R,G,B,alpha])
      
    return list_info

# Print of the table with the information I was interested, using the tabulate library.
def printa_tabule(info):
    headers = ["name","height","width","greyscale","R","G","B","ALPHA"]
    print(tabulate.tabulate(info,headers,tablefmt="grid"))

def main():
     
    position_files= "" ## INSERIRE Path della Posizione della directory "file" 
    informations_of_files, exist = iteration_files(position_files)
    recap(position_files,informations_of_files, exist)

    directory_images = position_files + "/images"
    list_of_images = os.listdir(directory_images)
    list_info = create_list_info(list_of_images,directory_images)
    printa_tabule( list_info ) 

if __name__ == "__main__":
    main()
